<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/16-stable-diffusion.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/16-stable-diffusion.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/16-stable-diffusion.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

In [1]:
# Import dependencies
import os
from IPython.display import Image

### Connect to EvaDB

In [2]:
%pip install --quiet "evadb[document,notebook]"
%pip install replicate
%pip install diffusers
%pip install accelerate
%pip install importlib
%pip install argparse
%pip install omegaconf==2.1
import evadb
cursor = evadb.connect().cursor()

DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A p

DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
DEPRECATION: omegaconf 2.1.0 has a non-standard dependency specifier PyYA

### Get Input Prompt from User

In [3]:
# to collect all user prompts
def get_user_input():
    print('Welcome to EvaDB!')
    print('Enter your image zip file location.')
    print('========================================')
    prompts = []
    prompt=None

    # receive all prompts from user
    prompt = input(
        'Enter prompt: '
    ).strip()

    return prompt

In [4]:
# getting user input
prompt = get_user_input()

Welcome to EvaDB!
Enter your image zip file location.
Enter prompt: /Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora


### Set API Token Environment Variable

In [5]:
# to set the replicate API token environment variable
def set_replicate_token() -> None:
    key = input('Enter your Replicate API Token: ').strip()

    try:
        os.environ['REPLICATE_API_TOKEN'] = key
        print('Environment variable set successfully.')
    except Exception as e:
        print("❗️ Session ended with an error.")
        print(e)
        print("===========================================")

In [6]:
# setting api token as env variable
set_replicate_token()

Enter your Replicate API Token: r8_Zunf00BeIdS4OareDu4OnYze7qZHab41memU3
Environment variable set successfully.


### Load the Stable Diffusion UDF

In [64]:
# set up the stable diffusion UDF available at functions/stable_diffusion.py
cursor.query("""CREATE FUNCTION IF NOT EXISTS StableDiffusionLoRA
            IMPL  '../evadb/functions/stable_diffusion_lora.py';
                """).execute()

### Create Table

In [8]:
# delete the table if it already exists
cursor.query("""DROP TABLE IF EXISTS MyImages
                """).execute()


### Load Prompts into Table

In [9]:
import PIL.Image
for file_number, file_entry in enumerate(os.scandir(prompt)):
    # filter out non-images first
    print(file_entry)
    print(file_entry.path)
    cursor.query(f"""LOAD IMAGE '{file_entry.path}' INTO MyImages;""").execute()

<DirEntry 'download.jpeg'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/download.jpeg
<DirEntry 'ebdd5c56c4ef74307c27a09f13f9cba395233951.jpg'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/ebdd5c56c4ef74307c27a09f13f9cba395233951.jpg
<DirEntry 'nightmare.webp'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/nightmare.webp
<DirEntry '387-c7e4d56f-1.jpg'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/387-c7e4d56f-1.jpg
<DirEntry 'images.jpeg'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/images.jpeg
<DirEntry 'gothic_art_and_architecture_7.jpg'>
/Users/kacylombard/Desktop/evadb/evadb/test/integration_tests/long/functions/test_images_for_lora/gothic_art_and_architecture_7.jpg
<DirEntry 'The-Crucifixion-and-Last-Judgement-diptych-by

### Run Stable Diffusion on the Prompts

In [65]:
table = cursor.query("SELECT StableDiffusionLoRA(*) FROM MyImages GROUP BY 'name 19';").df()


<bound method DataFrame.info of     _row_id                                               name  \
0         1  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
1         2  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
2         3  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
3         4  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
4         5  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
5         6  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
6         7  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
7         8  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
8         9  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
9        10  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
10       11  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
11       12  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
12       13  /Users/kacylombard/Desktop/evadb/evadb/test/in...   
13       14  /Users/kacylombard/Desktop/evad